In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns


Load Data

In [ ]:
def load_dataframes():
    """Load all sensor data into pandas DataFrames"""
    
    BASE_PATH = Path('/Users/olli/Desktop/Uni/Predictive Analytics/Predictive-Analytics-Project/data/raw/predictive_analytics_data_testdb')
    
    # Define paths for sensor data
    DATA_PATHS = {
        'electric_sensor': BASE_PATH / 'sensor_data/household_sensors_electric.csv',
        'gas_sensor': BASE_PATH / 'sensor_data/household_sensors_gas.csv',
        'room_sensors': BASE_PATH / 'sensor_data/room_and_appliance_sensors_room.csv',
        'tempprobe_sensors': BASE_PATH / 'sensor_data/room_and_appliance_sensors_tempprobe.csv',
        #'weather': BASE_PATH / 'sensor_data/weather_complete.csv'
    }
    
    # Load each dataset
    dataframes = {}
    for name, path in DATA_PATHS.items():
        try:
            df = pd.read_csv(path)
            # Convert timestamp columns to datetime
            if 'timestamp' in df.columns:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
            dataframes[name] = df
            print(f"Loaded {name}: {df.shape[0]} rows, {df.shape[1]} columns")
        except Exception as e:
            print(f"Error loading {name}: {str(e)}")
    
    return dataframes

# Load all dataframes
dfs = load_dataframes()
    

Time Series Analysis